In [1]:
import requests
import bs4
import pandas as pd
from datetime import datetime
import time
from math import floor

In [2]:
year = 2019
url = 'https://www.baseball-reference.com/leagues/MLB/' + str(year) + '-schedule.shtml'

In [3]:
res = requests.get(url)
soup=bs4.BeautifulSoup(res.text, 'html.parser')

In [4]:
res

<Response [200]>

In [5]:
content = soup.find('div',{'class': "section_content"})

In [6]:
len(content('div'))

185

In [7]:
YEAR = []
DATES_ENGLISH = []
DATES_ALPHA = []
AWAYTEAMFULL = []
AWAYTEAMABRR = []
HOMETEAMFULL = []
HOMETEAMABRR = []
AWAYTEAMSCORE = []
HOMETEAMSCORE = []

In [14]:
start = time.perf_counter()

GAMEKEYS = []
YEAR = []
MONTHS =[]
DAYOFWEEK = []
DATES_ALPHA = []
AWAYTEAMFULL = []
AWAYTEAMABRR = []
HOMETEAMFULL = []
HOMETEAMABRR = []
AWAYTEAMSCORE = []
HOMETEAMSCORE = []
STARTIME = []
ATTENDANCE = []
VENUE = []
DURATION = []
DOUBLEHEADER = []

for divs in content('div')[:5]:
    h = divs.find('h3')
    games = divs('p', {'class':'game'})
    for p in games:
        a1 = p('a')[0]
        a2 = p('a')[1]
        
        boxscore = p.find('em')
        boxscoreUrl = boxscore.find('a')['href']
        
        boxUrl = 'https://www.baseball-reference.com/' + boxscoreUrl
        
        boxRes = requests.get(boxUrl)
        boxSoup=bs4.BeautifulSoup(boxRes.text, 'html.parser')
        
        boxMeta = boxSoup.find('div',{'class':'scorebox_meta'})
        
        for bm in boxMeta('div'):
            for bms in bm('strong'):

                if bms.text.strip() == 'Attendance':
                    attendance = int(bm.text.split(': ')[1].replace(',',''))
                elif bms.text.strip() == 'Venue':
                    venue = bm.text.split(': ')[1]
                elif bms.text.strip() == 'Game Duration':
                    gameDur = bm.text.split(': ')[1]
        
        
        date = datetime.strptime(h.text, '%A, %B %d, %Y').strftime("%Y-%m-%d")
        homeTeam = a2['href'].split('/')[2]
        awayTeam = a1['href'].split('/')[2]
        doubleHeader = boxscoreUrl.replace('.shtml','')[-1]
        gameKey = date.replace('-','') + homeTeam + awayTeam + doubleHeader
        
        dayOfWeek= h.text.split(',')[0]
        month = h.text.split()[1]
        
        
        GAMEKEYS.append(gameKey)
        YEAR.append(year)
        MONTHS.append(month)
        DAYOFWEEK.append(dayOfWeek)
        DATES_ALPHA.append(date)
        AWAYTEAMFULL.append(a1.text)
        HOMETEAMFULL.append(a2.text)
        AWAYTEAMABRR.append(awayTeam)
        HOMETEAMABRR.append(homeTeam)
        AWAYTEAMSCORE.append(int(p.text.split('(')[1].split(')')[0]))
        HOMETEAMSCORE.append(int(p.text.split('(')[2].split(')')[0]))
        STARTIME.append(boxMeta('div')[1].text.split(': ')[1])
        ATTENDANCE.append(attendance)
        VENUE.append(venue)
        DURATION.append(gameDur)
        DOUBLEHEADER.append(doubleHeader)
        
        elapsed_ = time.perf_counter() - start
        hours   = floor(elapsed_ / 60**2)
        minutes = floor((elapsed_ - hours*60**2) / 60)
        seconds = floor(elapsed_ - hours*60**2 -minutes*60)
        
        print(date, ' - ', awayTeam + ' @ ' + homeTeam, ' - ',  'Elapsed Time:',  hours, 'hours', minutes, 'minutes', seconds, 'seconds')
    

columns = ['gameKey', 'date','doubleHeader','year','month','dayOfWeek', 'startTime', 'homeTeam', 'homeTeamAbbr', 'awayTeam', 'awayTeamAbbr', 'homeTeamScore', 'awayTeamScore', 'venue', 'attendance', 'gameDuration']
dataDict = {}
dataDict['gameKey'] = GAMEKEYS
dataDict['date'] = DATES_ALPHA
dataDict['doubleHeader'] = DOUBLEHEADER
dataDict['year'] = YEAR
dataDict['month'] = MONTHS
dataDict['dayOfWeek'] = DAYOFWEEK
dataDict['startTime'] = STARTIME
dataDict['homeTeam'] = HOMETEAMFULL
dataDict['homeTeamAbbr'] = HOMETEAMABRR
dataDict['awayTeam'] = AWAYTEAMFULL
dataDict['awayTeamAbbr'] = AWAYTEAMABRR
dataDict['homeTeamScore'] = HOMETEAMSCORE
dataDict['awayTeamScore'] = AWAYTEAMSCORE
dataDict['attendance'] = ATTENDANCE
dataDict['venue'] = VENUE
dataDict['gameDuration'] = DURATION

df = pd.DataFrame(dataDict, columns=columns)
df

2019-03-20  -  SEA @ OAK  -  Elapsed Time: 0 hours 0 minutes 0 seconds
2019-03-21  -  SEA @ OAK  -  Elapsed Time: 0 hours 0 minutes 1 seconds
2019-03-28  -  PIT @ CIN  -  Elapsed Time: 0 hours 0 minutes 1 seconds
2019-03-28  -  CHW @ KCR  -  Elapsed Time: 0 hours 0 minutes 2 seconds
2019-03-28  -  ARI @ LAD  -  Elapsed Time: 0 hours 0 minutes 2 seconds
2019-03-28  -  COL @ MIA  -  Elapsed Time: 0 hours 0 minutes 3 seconds
2019-03-28  -  STL @ MIL  -  Elapsed Time: 0 hours 0 minutes 3 seconds
2019-03-28  -  CLE @ MIN  -  Elapsed Time: 0 hours 0 minutes 4 seconds
2019-03-28  -  BAL @ NYY  -  Elapsed Time: 0 hours 0 minutes 4 seconds
2019-03-28  -  LAA @ OAK  -  Elapsed Time: 0 hours 0 minutes 5 seconds
2019-03-28  -  ATL @ PHI  -  Elapsed Time: 0 hours 0 minutes 5 seconds
2019-03-28  -  SFG @ SDP  -  Elapsed Time: 0 hours 0 minutes 6 seconds
2019-03-28  -  BOS @ SEA  -  Elapsed Time: 0 hours 0 minutes 6 seconds
2019-03-28  -  HOU @ TBR  -  Elapsed Time: 0 hours 0 minutes 7 seconds
2019-0

,gameKey,date,doubleHeader,year,month,dayOfWeek,startTime,homeTeam,homeTeamAbbr,awayTeam,awayTeamAbbr,homeTeamScore,awayTeamScore,venue,attendance,gameDuration
0,20190320OAKSEA0,2019-03-20,0,2019,March,Wednesday,6:36 p.m. Local,Oakland Athletics,OAK,Seattle Mariners,SEA,7,9,Tokyo Dome,45787,3:24
1,20190321OAKSEA0,2019-03-21,0,2019,March,Thursday,6:35 p.m. Local,Oakland Athletics,OAK,Seattle Mariners,SEA,4,5,Tokyo Dome,46451,4:27
2,20190328CINPIT0,2019-03-28,0,2019,March,Thursday,4:11 p.m. Local,Cincinnati Reds,CIN,Pittsburgh Pirates,PIT,5,3,Great American Ball Park,44049,2:54
3,20190328KCRCHW0,2019-03-28,0,2019,March,Thursday,5:01 p.m. Local,Kansas City Royals,KCR,Chicago White Sox,CHW,5,3,Kauffman Stadium,31675,3:12
4,20190328LADARI0,2019-03-28,0,2019,March,Thursday,1:12 p.m. Local,Los Angeles Dodgers,LAD,Arizona D'Backs,ARI,12,5,Dodger Stadium,53086,2:49
5,20190328MIACOL0,2019-03-28,0,2019,March,Thursday,4:12 p.m. Local,Miami Marlins,MIA,Colorado Rockies,COL,3,6,Marlins Park,25423,2:55
6,20190328MILSTL0,2019-03-28,0,2019,March,Thursday,1:10 p.m. Local,Milwaukee Brewers,MIL,St. Louis Cardinals,STL,5,4,Miller Park,45304,2:36
7,20190328MINCLE0,2019-03-28,0,2019,March,Thursday,3:17 p.m. Local,Minnesota Twins,MIN,Cleveland Indians,CLE,2,0,Target Field,39519,2:18
8,20190328NYYBAL0,2019-03-28,0,2019,March,Thursday,1:12 p.m. Local,New York Yankees,NYY,Baltimore Orioles,BAL,7,2,Yankee Stadium III,46928,3:06
9,20190328OAKLAA0,2019-03-28,0,2019,March,Thursday,1:10 p.m. Local,Oakland Athletics,OAK,Los Angeles Angels,LAA,4,0,Oakland-Alameda County Coliseum,22691,2:18


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gameKey        39 non-null     object
 1   date           39 non-null     object
 2   doubleHeader   39 non-null     object
 3   year           39 non-null     int64 
 4   month          39 non-null     object
 5   dayOfWeek      39 non-null     object
 6   startTime      39 non-null     object
 7   homeTeam       39 non-null     object
 8   homeTeamAbbr   39 non-null     object
 9   awayTeam       39 non-null     object
 10  awayTeamAbbr   39 non-null     object
 11  homeTeamScore  39 non-null     int64 
 12  awayTeamScore  39 non-null     int64 
 13  venue          39 non-null     object
 14  attendance     39 non-null     int64 
 15  gameDuration   39 non-null     object
dtypes: int64(4), object(12)
memory usage: 5.0+ KB


In [89]:
for divs in content('div')[:5]:
    h = divs.find('h3')
    games = divs('p', {'class':'game'})
    for p in games:
        a1 = p('a')[0]
        a2 = p('a')[1]
        
        boxscore = p.find('em')
        boxUrl = 'https://www.baseball-reference.com/' + boxscore.find('a')['href']
        
        boxRes = requests.get(boxUrl)
        boxSoup=bs4.BeautifulSoup(boxRes.text, 'html.parser')
        
        boxMeta = boxSoup.find('div',{'class':'scorebox_meta'})
        print(boxUrl, 'StartTime: ', boxMeta('div')[1].text.split(': ')[1], int(boxMeta('div')[2].text.split(': ')[1].replace(',','')), boxMeta('div')[3].text.split(': ')[1], boxMeta('div')[4].text.split(': ')[1] )
        
        #print(boxscore.find('a')['href'])

https://www.baseball-reference.com//boxes/OAK/OAK201903200.shtml StartTime:  5:35 p.m. ET 45787 Tokyo Dome 3:24
https://www.baseball-reference.com//boxes/OAK/OAK201903210.shtml StartTime:  5:35 p.m. ET 46451 Tokyo Dome 4:27
https://www.baseball-reference.com//boxes/CIN/CIN201903280.shtml StartTime:  4:10 p.m. ET 44049 Great American Ball Park 2:54
https://www.baseball-reference.com//boxes/KCA/KCA201903280.shtml StartTime:  4:15 p.m. ET 31675 Kauffman Stadium 3:12
https://www.baseball-reference.com//boxes/LAN/LAN201903280.shtml StartTime:  4:10 p.m. ET 53086 Dodger Stadium 2:49
https://www.baseball-reference.com//boxes/MIA/MIA201903280.shtml StartTime:  4:10 p.m. ET 25423 Marlins Park 2:55
https://www.baseball-reference.com//boxes/MIL/MIL201903280.shtml StartTime:  2:10 p.m. ET 45304 Miller Park 2:36
https://www.baseball-reference.com//boxes/MIN/MIN201903280.shtml StartTime:  4:10 p.m. ET 39519 Target Field 2:18
https://www.baseball-reference.com//boxes/NYA/NYA201903280.shtml StartTime:

In [74]:
boxUrl = 'https://www.baseball-reference.com/' + boxscore.find('a')['href']

boxRes = requests.get(boxUrl)
boxSoup=bs4.BeautifulSoup(boxRes.text, 'html.parser')

boxMeta = boxSoup.find('div',{'class':'scorebox_meta'})

boxMeta('div')[1].text

'Start Time: 5:35 p.m. ET'

In [92]:
boxUrl = 'https://www.baseball-reference.com/boxes/PIT/PIT200004040.shtml'
        
boxRes = requests.get(boxUrl)
boxSoup=bs4.BeautifulSoup(boxRes.text, 'html.parser')

boxMeta = boxSoup.find('div',{'class':'scorebox_meta'})

In [118]:
for bm in boxMeta('div'):
    for bms in bm('strong'):
        print(bms.text.strip())
        
        if bms.text.strip() == 'Attendance':
            attendance = int(bm.text.split(': ')[1].replace(',',''))
        elif bms.text.strip() == 'Venue':
            venue = bm.text.split(': ')[1]
        elif bms.text.strip() == 'Game Duration':
            gameDur = bm.text.split(': ')[1]
    
  
print(attendance, venue, gameDur)     

Recheduled From
Attendance
Venue
Game Duration
54399 Three Rivers Stadium 2:51


In [17]:
def reset_lists():
    GAMEKEYS = []
    YEAR = []
    MONTHS =[]
    DAYOFWEEK = []
    DATES_ALPHA = []
    AWAYTEAMFULL = []
    AWAYTEAMABRR = []
    HOMETEAMFULL = []
    HOMETEAMABRR = []
    AWAYTEAMSCORE = []
    HOMETEAMSCORE = []
    STARTIME = []
    ATTENDANCE = []
    VENUE = []
    DURATION = []
    DOUBLEHEADER = []

In [18]:
reset_lists()

In [19]:
GAMEKEYS

['20190320OAKSEA0',
 '20190321OAKSEA0',
 '20190328CINPIT0',
 '20190328KCRCHW0',
 '20190328LADARI0',
 '20190328MIACOL0',
 '20190328MILSTL0',
 '20190328MINCLE0',
 '20190328NYYBAL0',
 '20190328OAKLAA0',
 '20190328PHIATL0',
 '20190328SDPSFG0',
 '20190328SEABOS0',
 '20190328TBRHOU0',
 '20190328TEXCHC0',
 '20190328TORDET0',
 '20190328WSNNYM0',
 '20190329LADARI0',
 '20190329MIACOL0',
 '20190329MILSTL0',
 '20190329OAKLAA0',
 '20190329SDPSFG0',
 '20190329SEABOS0',
 '20190329TBRHOU0',
 '20190329TORDET0',
 '20190330KCRCHW0',
 '20190330LADARI0',
 '20190330MIACOL0',
 '20190330MILSTL0',
 '20190330MINCLE0',
 '20190330NYYBAL0',
 '20190330OAKLAA0',
 '20190330PHIATL0',
 '20190330SDPSFG0',
 '20190330SEABOS0',
 '20190330TBRHOU0',
 '20190330TEXCHC0',
 '20190330TORDET0',
 '20190330WSNNYM0']